# Library import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from keras import backend as K
from keras import optimizers
from keras.layers import Input, Dense, Lambda, Layer, Add, Multiply
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.losses import mean_squared_error
from keras import layers as KL

import uproot
import numpy
import matplotlib.pyplot
import pandas as pd
from pathlib import Path

from keras.layers import Input, Dense, Lambda, Layer, LeakyReLU, ReLU
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras import optimizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, BaseLogger, \
TerminateOnNaN, Callback, ModelCheckpoint, LambdaCallback

# from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
# from sklearn.model_selection import train_test_split

from keras.constraints import max_norm
from sklearn.externals.joblib import dump, load

from numpy.random import seed
import time

import json
import pickle
import os
import shutil

import random as rn

np.random.seed(42)
# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
rn.seed(12345)

# import ray
# from ray.tune.integration.keras import TuneReporterCallback
# from ray import tune
# from ray.tune.schedulers import AsyncHyperBandScheduler, ASHAScheduler, PopulationBasedTraining
import random
import csv

from config import *
from vae_utility import *

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(42)
# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
rn.seed(12345)

Using TensorFlow backend.
/home/roberto/anaconda3/envs/exotica_ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/roberto/anaconda3/envs/exotica_ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/roberto/anaconda3/envs/exotica_ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/h

# Define the features weight:

In the components_dict put the weight for each feature: higher the value higher
the importance of the variable. The model try to reconstruct better the feature with higher value

for the bump the 'mt' and 'mct2' are the most important features

with "0" as weight the variabble is still used during the train but their information is masked. 
We refer to this variables as the PASSIVELY used vriable opposing to the ACTIVELY used variables.

Setting 0 to some variable is different to completely exclude the feature from the train beacause in this latter case the original space dimension decrease. The latent dimension that is smaller than the original space dimension can vary from 1 to 10. If you decrease the original dimension to only 3 feat for example the latent dimension is upper bounded (at most equal to three)


In [2]:
# Here we load the numpy data (already splitted in train-val-test) 
# We limit to the model-independent case but from the script (030-033) also
# the model dependent case is avaible

back = np.load(train_val_test + '/background_train.npy')
back_val = np.load(train_val_test + '/background_val.npy')

# Remind that this data also include the weights of the data events (010 python script) so
#it is needed to remove the weights from the train-val data: the last column are the weight we use 
# during the signal selection (test-time)
train = back[:,:-1]
val = back_val[:,:-1]

# Put here the weight you want to use
components_dict = {
    'met':1,
    'mt':1,
    'mbb':2,
    'mct2':1,
    'mlb1':1,
    'lep1Pt':1,
    'nJet30':1,
    'nBJet30_MV2c10':1
    }

#Infer the component actively used during the train
selected_components = []
for k,v in components_dict.items():
    if components_dict[k] != 0:
        selected_components.append(k)
        
#Infer the weights
weights = []
for k,v in components_dict.items():
    weights.append(v)

selected_idx = [cols.index(component) for component in selected_components]

if 0 in weights:
    ind_w = [x if x != 0 else 1 for x in weights]
else:
    ind_w = weights

w = weights

In [3]:
# This variable are quite fixed, only with modification to the samples (exclude some feature)
# they need to be update
original_dim = train.shape[1]
Nf_lognorm = 6
Nf_PDgauss = 2
# More flexible parameters, change to affect the model configuration
# This parameter are a good starting point tested on (30-30-40) data split
# Change to try different model configuration
intermediate_dim=50
act_fun='relu'
latent_dim=2
kernel_max_norm=500
lr=0.003
epochs=2000 
weight_KL_loss=0.6
batch_size=200
#####################################################################################################
name = 'bump_feat_{}_ld_{}'.format('_'.join([str(x) for x in selected_idx]), latent_dim)

try:
    os.makedirs(model_results_bump_single + name)
except:
    pass
    
with open(model_results_bump_single + name + '/' + 'comps_dict.pickle', 'wb') as handle:
    pickle.dump(components_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

print("\x1b[31m\" model {} with selected component: {} and weight {} in loss, individual weights{}""\x1b[0m"\
      .format(name, selected_components, weights, ind_w))

" model bump_feat_0_1_2_3_4_5_6_7_ld_2 with selected component: ['met', 'mt', 'mbb', 'mct2', 'mlb1', 'lep1Pt', 'nJet30', 'nBJet30_MV2c10'] and weight [1, 1, 2, 1, 1, 1, 1, 1] in loss, individual weights[1, 1, 2, 1, 1, 1, 1, 1]


# Technical details:

Useful to understand but not a strict requirement

Here we define all the function that manage the loss used during the model train:

The model final output are the parameters (as the mean and the standard deviation for example) of the 
probability distribution associated with each physical event. Each event is characterized from 
its features (met, mbb, mct2,....nBjet30 and so on). The model learn to associate to each events
a distribution to describe each variable. These distribution represent the probability of the specific event to assume a range of possible value.

These distribution are the result of the reconstruction step. Indeed, before the reconstruction 
can happen, the model also compress the events from its original space (8 dimensional) to 
a lower space (Latent space): compress the input during the encoding step. So,, to reconnect to the final step (reconstruction), the model starting from this compressed rapresentation try to reconstruct the original feature. In this way, first compressing and then trying to reconstruct, the model learn to model the features distribution

The model so try to maximize the input data likelihood (actually minimize the negative likelihoog) searching for the optimal distribution parameters to associate with each event. In the next cell the RecoProb_forVae actually is the computation of the likelihood given some inputs e the relative distribution parameters. Instead, the KL_loss_forVAE, is a part of the total loss (Total loss =
KL_loss + ReconProb_Loss), used to get an optimal latent space. For what concern the Bump Searches the Recon Loss is the part where mostly focus on. 


The RecoProb Loss is the sum of the individual features losses: each event has its individual losses for every features, and from their sum we obtain the reco_prob_loss. To monitor the individual loss (we are particularly interested in the 'met' and 'mct2') other function as the "IndividualRecoProb_forVAE_lognorm_1" are defined. Each of these function report the individual feature loss.

<font color='red'>
Note the w parameters in these loss
</font>

these w parameter are those defined before for each variables <font color='red'> (components_dict) </font>

<img src="images_jup/scheme_work.png">

In [4]:
################################# KL LOSS ############################
# Loss for the latent space optimization
def KL_loss_forVAE(mu, sigma, mu_prior, sigma_prior):
    kl_loss = K.tf.multiply(K.square(sigma), K.square(sigma_prior))
    kl_loss += K.square(K.tf.divide(mu_prior - mu, sigma_prior))
    kl_loss += K.log(K.tf.divide(sigma_prior, sigma)) -1
    return 0.5 * K.sum(kl_loss, axis=-1)

########################## RecoProb Loss #############################
# This is the sum of each individual feature loss (defined Below)
def RecoProb_forVAE(x, par1, par2, par3, w):

    N = 0
    nll_loss = 0

    if Nf_lognorm != 0:

        for i in range(Nf_lognorm):

            #Log-Normal distributed variables
            mu = par1[:,i:i+1]
            sigma = par2[:,i:i+1]
            fraction = par3[:,i:i+1]
            x_clipped = K.clip(x[:,i:i+1], clip_x_to0, 1e8)
            single_NLL = K.tf.where(K.less(x[:,i:i+1], clip_x_to0),
                                    -K.log(fraction),
                                        -K.log(1-fraction)
                                        + K.log(sigma)
                                        + K.log(x_clipped)
                                        + 0.5*K.square(K.tf.divide(K.log(x_clipped) - mu, sigma)))
            nll_loss += K.sum(w[i]*single_NLL, axis=-1)

        N += Nf_lognorm

    if Nf_PDgauss != 0:

        for i in range(N, N+Nf_PDgauss):

            mu = par1[:,i:i+1]
            sigma = par2[:,i:i+1]
            norm_xp = K.tf.divide(x[:,i:i+1] + 0.5 - mu, sigma)
            norm_xm = K.tf.divide(x[:,i:i+1] - 0.5 - mu, sigma)
            sqrt2 = 1.4142135624
            single_LL = 0.5*(K.tf.erf(norm_xp/sqrt2) - K.tf.erf(norm_xm/sqrt2))

            norm_0 = K.tf.divide(-0.5 - mu, sigma)
            aNorm = 1 + 0.5*(1 + K.tf.erf(norm_0/sqrt2))
            single_NLL = -K.log(K.clip(single_LL, 1e-10, 1e40)) -K.log(aNorm)

            nll_loss += K.sum(w[i]*single_NLL, axis=-1)

    return nll_loss

#######################################################################################
########################## Definition of individual loss for each variable ############
#######################################################################################

def IndividualRecoProb_forVAE_lognorm_1(x, par1, par2, par3, w):
    N = 0
    nll_loss = 0

    mu = par1[:,:1]
    sigma = par2[:,:1]
    fraction = par3[:,:1]
    x_clipped = K.clip(x[:,:1], clip_x_to0, 1e8)
    single_NLL = K.tf.where(K.less(x[:,:1], clip_x_to0),
                            -K.log(fraction),
                                -K.log(1-fraction)
                                + K.log(sigma)
                                + K.log(x_clipped)
                                + 0.5*K.square(K.tf.divide(K.log(x_clipped) - mu, sigma)))
    nll_loss += K.sum(w*single_NLL, axis=-1)

    return nll_loss

def IndividualRecoProb_forVAE_lognorm_2(x, par1, par2, par3, w):
    N = 0
    nll_loss = 0

    mu = par1[:,1:2]
    sigma = par2[:,1:2]
    fraction = par3[:,1:2]
    x_clipped = K.clip(x[:,1:2], clip_x_to0, 1e8)
    single_NLL = K.tf.where(K.less(x[:,1:2], clip_x_to0),
                            -K.log(fraction),
                                -K.log(1-fraction)
                                + K.log(sigma)
                                + K.log(x_clipped)
                                + 0.5*K.square(K.tf.divide(K.log(x_clipped) - mu, sigma)))
    nll_loss += K.sum(w*single_NLL, axis=-1)

    return nll_loss

def IndividualRecoProb_forVAE_lognorm_3(x, par1, par2, par3, w):
    N = 0
    nll_loss = 0

    mu = par1[:,2:3]
    sigma = par2[:,2:3]
    fraction = par3[:,2:3]
    x_clipped = K.clip(x[:,2:3], clip_x_to0, 1e8)
    single_NLL = K.tf.where(K.less(x[:,2:3], clip_x_to0),
                            -K.log(fraction),
                                -K.log(1-fraction)
                                + K.log(sigma)
                                + K.log(x_clipped)
                                + 0.5*K.square(K.tf.divide(K.log(x_clipped) - mu, sigma)))
    nll_loss += K.sum(w*single_NLL, axis=-1)

    return nll_loss

def IndividualRecoProb_forVAE_lognorm_4(x, par1, par2, par3, w):
    N = 0
    nll_loss = 0

    mu = par1[:,3:4]
    sigma = par2[:,3:4]
    fraction = par3[:,3:4]
    x_clipped = K.clip(x[:,3:4], clip_x_to0, 1e8)
    single_NLL = K.tf.where(K.less(x[:,3:4], clip_x_to0),
                            -K.log(fraction),
                                -K.log(1-fraction)
                                + K.log(sigma)
                                + K.log(x_clipped)
                                + 0.5*K.square(K.tf.divide(K.log(x_clipped) - mu, sigma)))
    nll_loss += K.sum(w*single_NLL, axis=-1)

    return nll_loss

def IndividualRecoProb_forVAE_lognorm_5(x, par1, par2, par3, w):
    N = 0
    nll_loss = 0

    mu = par1[:,4:5]
    sigma = par2[:,4:5]
    fraction = par3[:,4:5]
    x_clipped = K.clip(x[:,4:5], clip_x_to0, 1e8)
    single_NLL = K.tf.where(K.less(x[:,4:5], clip_x_to0),
                            -K.log(fraction),
                                -K.log(1-fraction)
                                + K.log(sigma)
                                + K.log(x_clipped)
                                + 0.5*K.square(K.tf.divide(K.log(x_clipped) - mu, sigma)))
    nll_loss += K.sum(w*single_NLL, axis=-1)

    return nll_loss

def IndividualRecoProb_forVAE_lognorm_6(x, par1, par2, par3, w):
    N = 0
    nll_loss = 0

    mu = par1[:,5:6]
    sigma = par2[:,5:6]
    fraction = par3[:,5:6]
    x_clipped = K.clip(x[:,5:6], clip_x_to0, 1e8)
    single_NLL = K.tf.where(K.less(x[:,5:6], clip_x_to0),
                            -K.log(fraction),
                                -K.log(1-fraction)
                                + K.log(sigma)
                                + K.log(x_clipped)
                                + 0.5*K.square(K.tf.divide(K.log(x_clipped) - mu, sigma)))
    nll_loss += K.sum(w*single_NLL, axis=-1)

    return nll_loss


def individualRecoProb_forVAE_discrete_7(x, par1, par2, w):
    N = Nf_lognorm
    nll_loss = 0

    mu = par1[:,6:7]
    sigma = par2[:,6:7]
    norm_xp = K.tf.divide(x[:,6:7] + 0.5 - mu, sigma)
    norm_xm = K.tf.divide(x[:,6:7] - 0.5 - mu, sigma)
    sqrt2 = 1.4142135624
    single_LL = 0.5*(K.tf.erf(norm_xp/sqrt2) - K.tf.erf(norm_xm/sqrt2))

    norm_0 = K.tf.divide(-0.5 - mu, sigma)
    aNorm = 1 + 0.5*(1 + K.tf.erf(norm_0/sqrt2))
    single_NLL = -K.log(K.clip(single_LL, 1e-10, 1e40)) -K.log(aNorm)

    nll_loss += K.sum(w*single_NLL, axis=-1)

    return nll_loss

def individualRecoProb_forVAE_discrete_8(x, par1, par2, w):
    N = Nf_lognorm
    nll_loss = 0

    mu = par1[:,7:8]
    sigma = par2[:,7:8]
    norm_xp = K.tf.divide(x[:,7:8] + 0.5 - mu, sigma)
    norm_xm = K.tf.divide(x[:,7:8] - 0.5 - mu, sigma)
    sqrt2 = 1.4142135624
    single_LL = 0.5*(K.tf.erf(norm_xp/sqrt2) - K.tf.erf(norm_xm/sqrt2))

    norm_0 = K.tf.divide(-0.5 - mu, sigma)
    aNorm = 1 + 0.5*(1 + K.tf.erf(norm_0/sqrt2))
    single_NLL = -K.log(K.clip(single_LL, 1e-10, 1e40)) -K.log(aNorm)

    nll_loss += K.sum(w*single_NLL, axis=-1)

    return nll_loss

# Techinal details:

To implement the previous loss, we define some specific layer for the model

In [7]:
#######################  Layer to compute the KL Loss   ###################################
class CustomKLLossLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomKLLossLayer, self).__init__(**kwargs)

    def call(self, inputs):
        mu, sigma, mu_prior, sigma_prior = inputs
        return KL_loss_forVAE(mu, sigma, mu_prior, sigma_prior)

#######################  Layer to compute the RecoProb   ###################################
class CustomRecoProbLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomRecoProbLayer, self).__init__(**kwargs)

    def call(self, inputs):
        x, par1, par2, par3 = inputs
        return RecoProb_forVAE(x, par1, par2, par3, w = w)

######################### Layer to compute the individual RecoProb #############################################à
class CustomIndividualLogNorLayer_1(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomIndividualLogNorLayer_1, self).__init__(**kwargs)

    def call(self, inputs):
        x, par1, par2, par3 = inputs
        return IndividualRecoProb_forVAE_lognorm_1(x, par1, par2, par3, w = ind_w[0])

class CustomIndividualLogNorLayer_2(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomIndividualLogNorLayer_2, self).__init__(**kwargs)

    def call(self, inputs):
        x, par1, par2, par3 = inputs
        return IndividualRecoProb_forVAE_lognorm_2(x, par1, par2, par3, w = ind_w[1])


class CustomIndividualLogNorLayer_3(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomIndividualLogNorLayer_3, self).__init__(**kwargs)

    def call(self, inputs):
        x, par1, par2, par3 = inputs
        return IndividualRecoProb_forVAE_lognorm_3(x, par1, par2, par3, w = ind_w[2])

class CustomIndividualLogNorLayer_4(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomIndividualLogNorLayer_4, self).__init__(**kwargs)

    def call(self, inputs):
        x, par1, par2, par3 = inputs
        return IndividualRecoProb_forVAE_lognorm_4(x, par1, par2, par3, w = ind_w[3])

class CustomIndividualLogNorLayer_5(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomIndividualLogNorLayer_5, self).__init__(**kwargs)

    def call(self, inputs):
        x, par1, par2, par3 = inputs
        return IndividualRecoProb_forVAE_lognorm_5(x, par1, par2, par3, w = ind_w[4])

class CustomIndividualLogNorLayer_6(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomIndividualLogNorLayer_6, self).__init__(**kwargs)

    def call(self, inputs):
        x, par1, par2, par3 = inputs
        return IndividualRecoProb_forVAE_lognorm_6(x, par1, par2, par3, w = ind_w[5])

class CustomIndividualTruGauLayer_7(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomIndividualTruGauLayer_7, self).__init__(**kwargs)

    def call(self, inputs):
        x, par1, par2 = inputs
        return individualRecoProb_forVAE_discrete_7(x, par1, par2, w = ind_w[6])

class CustomIndividualTruGauLayer_8(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomIndividualTruGauLayer_8, self).__init__(**kwargs)

    def call(self, inputs):
        x, par1, par2 = inputs
        return individualRecoProb_forVAE_discrete_8(x, par1, par2, w = ind_w[7])

In [8]:
x_DNN_input = Input(shape=(original_dim,), name='Input')
hidden_1 = Dense(intermediate_dim, activation=act_fun, name='Encoder_h1')
aux = hidden_1(x_DNN_input)

hidden_2 = Dense(intermediate_dim, activation=act_fun, name='Encoder_h2')

aux = hidden_2(aux)

L_z_mean = Dense(latent_dim, name='Latent_mean')
T_z_mean = L_z_mean(aux)
L_z_sigma_preActivation = Dense(latent_dim, name='Latent_sigma_h')

aux = L_z_sigma_preActivation(aux)
L_z_sigma = Lambda(InverseSquareRootLinearUnit, name='Latent_sigma')
T_z_sigma = L_z_sigma(aux)

L_z_latent = Lambda(sampling, name='Latent_sampling')([T_z_mean, T_z_sigma])
decoder_h1 = Dense(intermediate_dim,
                   activation=act_fun,
                   kernel_constraint=max_norm(kernel_max_norm),
                   name='Decoder_h1')(L_z_latent)

decoder_h2 = Dense(intermediate_dim, activation=act_fun, name='Decoder_h2')(decoder_h1)

L_par1 = Dense(original_dim, name='Output_par1')(decoder_h2)

L_par2_preActivation = Dense(original_dim , name='par2_h')(decoder_h2)
L_par2 = Lambda(InverseSquareRootLinearUnit, name='Output_par2')(L_par2_preActivation)

L_par3_preActivation = Dense(Nf_lognorm, name='par3_h')(decoder_h2)
L_par3 = Lambda(ClippedTanh, name='Output_par3')(L_par3_preActivation)

fixed_input = Lambda(SmashTo0)(x_DNN_input)
h1_prior = Dense(1,
                 kernel_initializer='zeros',
                 bias_initializer='ones',
                 trainable=False,
                 name='h1_prior'
                )(fixed_input)

L_prior_mean = Dense(latent_dim,
                     kernel_initializer='zeros',
                     bias_initializer='zeros',
                     trainable=True,
                     name='L_prior_mean'
                    )(h1_prior)

L_prior_sigma_preActivation = Dense(latent_dim,
                                    kernel_initializer='zeros',
                                    bias_initializer='ones',
                                    trainable=True,
                                    name='L_prior_sigma_preAct'
                                   )(h1_prior)
L_prior_sigma = Lambda(InverseSquareRootLinearUnit, name='L_prior_sigma')(L_prior_sigma_preActivation)

params = KL.concatenate([T_z_mean, T_z_sigma, L_prior_mean, L_prior_sigma, L_par1, L_par2, L_par3], axis=1)

L_RecoProb_1 = CustomIndividualLogNorLayer_1(name='RecoNLL_met')([x_DNN_input,L_par1,
                                                              L_par2,L_par3])

L_RecoProb_2 = CustomIndividualLogNorLayer_2(name='RecoNLL_mt')([x_DNN_input,L_par1,
                                                              L_par2,L_par3])

L_RecoProb_3 = CustomIndividualLogNorLayer_3(name='RecoNLL_mbb')([x_DNN_input,L_par1,
                                                              L_par2,L_par3])

L_RecoProb_4 = CustomIndividualLogNorLayer_4(name='RecoNLL_mct2')([x_DNN_input,L_par1,
                                                              L_par2,L_par3])

L_RecoProb_5 = CustomIndividualLogNorLayer_5(name='RecoNLL_mlb1')([x_DNN_input,L_par1,
                                                              L_par2,L_par3])

L_RecoProb_6 = CustomIndividualLogNorLayer_6(name='RecoNLL_lep1Pt')([x_DNN_input,L_par1,
                                                               L_par2,L_par3])

L_RecoProb_7 = CustomIndividualTruGauLayer_7(name='RecoNLL_nJet30')([x_DNN_input,L_par1,
                                                              L_par2])

L_RecoProb_8 = CustomIndividualTruGauLayer_8(name='RecoNLL_nBJet30_MV2c10')([x_DNN_input,L_par1,
                                                              L_par2])


L_RecoProb = CustomRecoProbLayer(name='RecoNLL')([x_DNN_input, L_par1, L_par2, L_par3])
L_KLLoss = CustomKLLossLayer(name='KL')([T_z_mean, T_z_sigma, L_prior_mean, L_prior_sigma])
vae = Model(inputs=x_DNN_input, outputs=[L_KLLoss, L_RecoProb,
                                        L_RecoProb_1, L_RecoProb_2,
                                        L_RecoProb_3, L_RecoProb_4, L_RecoProb_5,
                                        L_RecoProb_6, L_RecoProb_7, L_RecoProb_8])

In [10]:
# def metric_wapper(layer):

#     N = 4*latent_dim

#     par1 = layer[:, N: N+original_dim]
#     N += original_dim

#     par2 = layer[:, N: N+original_dim]
#     N += original_dim

#     par3 = layer[:, N:N+Nf_lognorm]

#     mu = layer[:, :latent_dim]
#     sigma = layer[:, latent_dim: 2*latent_dim]
#     mu_prior = layer[:, 2*latent_dim: 3*latent_dim]
#     sigma_prior = layer[:, 3*latent_dim: 4*latent_dim]

#     def metric(y_true, y_pred):

#         KL = weight_KL_loss*KL_loss_forVAE(mu,sigma,mu_prior,sigma_prior)
#         RecoLoss = RecoProb_forVAE(y_true[:, :original_dim], par1, par2, par3, w=1)

#         return (K.mean(RecoLoss+KL))

#     return metric

# metric = metric_wapper(params)

# Model Compile:

Once the model is defined, we need to compile it ("vae.compile") specifing the loss
we want to use

In [11]:
np.random.seed(42)
# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
rn.seed(12345)

adam = optimizers.adam(lr)

vae.compile(optimizer=adam,
            loss=[IdentityLoss, IdentityLoss,
                  IdentityLoss, IdentityLoss,IdentityLoss,IdentityLoss,
                 IdentityLoss,IdentityLoss,IdentityLoss,IdentityLoss],
            loss_weights=[weight_KL_loss, 1.,
                          0, 0, 0, 0,
                         0, 0, 0, 0]
#                     , metrics=[metric]
           )

# Train the model:

with vae.fit we start the model training

The callback are simply some function used to monitor the progress of the model like 
the training loss and the validation loss (both the total that individual one)

each time the model ends a train epoch (he use all the taining data) if it is enhancing its performances, then wil be saved as the new best model.

In [ ]:
#name of the model will be saved
name

In [11]:
train_history = vae.fit(x=train, y=[train, train,
                                    train, train, train, train, train, train, train, train],
    validation_data = (val, [val, val,
                             val, val, val, val, val, val, val, val]),
    shuffle=True,
    epochs=epochs,

    batch_size=batch_size,
    callbacks =
                [TerminateOnNaN(),
                ModelCheckpoint(model_results_bump_single + '{}/vae_{}.h5'.format(name, name),
                                    monitor='val_loss',
                                    mode='auto', save_best_only=True,verbose=1,
                                    period=1),
                    EarlyStopping(monitor='val_loss', patience=50, verbose=1, min_delta=0.3),
                    ReduceLROnPlateau(monitor='val_loss',
                                      factor=0.8,
                                      patience=5,
                                      mode = 'auto',
                                      epsilon=0.01,
                                      cooldown=0,
                                      min_lr=9e-8,
                                      verbose=1),
                SaveAutoencoder(model_results_bump_single +'{}/autoencoder_{}.h5'.format(name, name),
                                    monitor='val_loss',
                                    mode='auto', save_best_only=True,verbose=1,
                                    period=1)
                                    ])

hist_df = pd.DataFrame(train_history.history)

# or save to csv:
hist_csv_file = model_results_bump_single +'{}/history_{}.csv'.format(name, name)
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

/home/roberto/anaconda3/envs/exotica_ray/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Train on 36564 samples, validate on 36563 samples
Epoch 1/2000
36564/36564 [==============================] - 5s 133us/step - loss: 2653.9817 - KL_loss: 1710.2410 - RecoNLL_loss: 1627.8370 - RecoNLL_met_loss: 10.9295 - RecoNLL_mt_loss: 6.0868 - RecoNLL_mbb_loss: 14.7078 - RecoNLL_mct2_loss: 1514.2329 - RecoNLL_mlb1_loss: 33.6473 - RecoNLL_lep1Pt_loss: 40.7155 - RecoNLL_nJet30_loss: 4.6393 - RecoNLL_nBJet30_MV2c10_loss: 2.8781 - val_loss: 103.6071 - val_KL_loss: 97.3060 - val_RecoNLL_loss: 45.2235 - val_RecoNLL_met_loss: 6.5786 - val_RecoNLL_mt_loss: 5.1240 - val_RecoNLL_mbb_loss: 11.7019 - val_RecoNLL_mct2_loss: 7.1744 - val_RecoNLL_mlb1_loss: 6.7314 - val_RecoNLL_lep1Pt_loss: 5.0045 - val_RecoNLL_nJet30_loss: 1.5030 - val_RecoNLL_nBJet30_MV2c10_loss: 1.4058

Epoch 00001: val_loss improved from inf to 103.60714, saving model to model_results/single_train/bump_feat_0_1_2_3_4_5_6_7_ld_4/vae_bump_feat_0_1_2_3_4_5_6_7_ld_4.h5
Epoch 00001: val_loss improved from inf to 103.60714, saving mod